In [ ]:
import pandas as pd
import os

diretorio = 'C:/Users/Henri.Souza/Desktop/Henri/VSCODE/projetos/cepas5anos'

csv_arquivos = [f for f in os.listdir(diretorio) if f.endswith('.csv')]

lista_dfs = []

for arquivo in csv_arquivos:
    caminho = os.path.join(diretorio, arquivo)
    
    df = pd.read_csv(
        caminho,
        encoding='latin1',
        sep=';',
        decimal=',',
        skiprows=1  
    )
    
    lista_dfs.append(df)

dados = pd.concat(lista_dfs, ignore_index=True)


In [70]:
dados.isnull().sum()

ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF                 0
FORNECEDOR               0
DOCUMENTO             3979
DATA                     0
DETALHAMENTO         36383
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0
dtype: int64

In [71]:
dados = dados[~dados['DOCUMENTO'].isnull()]

In [72]:
dados = dados[~dados['DETALHAMENTO'].isnull()]

In [73]:
dados.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DOCUMENTO            0
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
COD_DOCUMENTO        0
dtype: int64

In [74]:
dados.shape

(54798, 11)

In [75]:
dados['VALOR_REEMBOLSADO'] = pd.to_numeric(dados['VALOR_REEMBOLSADO'], errors= 'coerce')

In [80]:
dados['CNPJ_CPF'] = dados['CNPJ_CPF'].astype(str).replace(r'\D', '', regex = True)

In [107]:
dados['COMPETENCIA'] = pd.to_datetime(dados['ANO'].astype(str) + '/' + dados['MES'].astype(str), format = '%Y/%m')


In [108]:
dados['COMPETENCIA'] = dados['COMPETENCIA'].dt.to_period('M')


In [110]:
dados['TIPO_DESPESA']  = dados['TIPO_DESPESA'].str.strip().str.upper()

In [112]:
dados['DETALHAMENTO'] = dados['DETALHAMENTO'].str.strip().str.upper()

In [115]:
dados['COD_DOCUMENTO'] = pd.to_numeric(dados['COD_DOCUMENTO'], errors='coerce')

In [116]:
dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,COMPETENCIA
1,2018,1,ACIR GURGACZ,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",63764211000110,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,02/01/2018,"COMPANHIA AÉREA: AVIANCA, LOCALIZADOR: M2CJMM....",274.99,2088476,2018-01
2,2018,1,ACIR GURGACZ,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",63764211000110,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,02/01/2018,"COMPANHIA AÉREA: LATAM, LOCALIZADOR: SFEXYF. P...",1094.29,2088528,2018-01
3,2018,1,ACIR GURGACZ,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",63764211000110,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,24/01/2018,"COMPANHIA AÉREA: AZUL, LOCALIZADOR: V64HTR. PA...",1553.99,2088475,2018-01
4,2018,2,ACIR GURGACZ,"ALUGUEL DE IMÓVEIS PARA ESCRITÓRIO POLÍTICO, C...",00494802863,GILBERTO PISELO DO NASCIMENTO,002/18,01/02/2018,DESPESA COM PAGAMENTO DE ALUGUEL DE IMÓVEL PAR...,6000.00,2088477,2018-02
5,2018,2,ACIR GURGACZ,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR,27170469000125,Infiniti Comunicação,0019,07/02/2018,DESPESA COM DIVULGAÇÃO DA ATIVIDADE PARLAMENTA...,1000.00,2088478,2018-02
...,...,...,...,...,...,...,...,...,...,...,...,...
93843,2022,12,ZEQUINHA MARINHO,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",22052777000132,Exceller Tour,WIXHAI,06/12/2022,"COMPANHIA AÉREA: LATAM, LOCALIZADOR: WIXHAI. P...",2893.04,2191398,2022-12
93844,2022,12,ZEQUINHA MARINHO,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",22052777000132,Exceller Tour,WITOLM,09/12/2022,"COMPANHIA AÉREA: GOL, LOCALIZADOR: WITOLM. PAS...",1180.19,2192272,2022-12
93845,2022,12,ZEQUINHA MARINHO,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",22052777000132,Exceller Tour,THPKVQ,20/12/2022,"COMPANHIA AÉREA: TAM, LOCALIZADOR: THPKVQ. PAS...",2671.90,2192274,2022-12
93846,2022,12,ZEQUINHA MARINHO,"PASSAGENS AÉREAS, AQUÁTICAS E TERRESTRES NACIO...",22052777000132,Exceller Tour,QNN9HX,21/12/2022,"COMPANHIA AÉREA: AZUL, LOCALIZADOR: QNN9HX. PA...",1334.31,2192244,2022-12
